In [1]:
import os
if not "experiments" in os.getcwd():
    %cd -q ../../project/parallel_synthesis/experiments 

In [2]:
from copy import copy

import torch
from protera_stability.config.lazy import LazyCall as L
from protera_stability.config.instantiate import instantiate
from protera_stability.config.common.mlp import mlp_esm

In [3]:
from protera_stability.train import get_cfg, setup_diversity, setup_data, setup_train

## Setup experiement, data and training cfg

In [4]:
exp_params = {
    "diversity_cutoff": 0.8, 
    "random_percent": 0.15,
    "sampling_method": "diversity",
    "experiment_name": "example"
}

cfg = get_cfg(args = {})
cfg = setup_diversity(cfg, **exp_params)
mlp_esm.n_units = 2048
mlp_esm.act = L(torch.nn.GELU)()
cfg.model = mlp_esm
cfg.model

{'n_in': 1280, 'n_units': 2048, 'n_layers': 3, 'act': {'_target_': <class 'torch.nn.modules.activation.GELU'>}, 'drop_p': 0.7, 'last_drop': False, '_target_': <class 'protera_stability.models.ProteinMLP'>}

In [5]:
cfg = setup_data(cfg)
cfg = setup_train(cfg)

In [6]:
cfg.keys()

dict_keys(['trainer_params', 'output_dir', 'random_split', 'experiment', 'model', 'dataloader', 'optim', 'scheduler'])

In [7]:
from protera_stability.train import do_train, do_test 

## Run Training

### Add specific callbacks

In [8]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

stop_r2_reached = L(EarlyStopping)(
        monitor="valid/r2",
        patience=1,
        check_on_train_epoch_end=False,
        stopping_threshold=0.72,
        mode="max",
    )
cbs = cfg.trainer_params["callbacks"]
cbs += [stop_r2_reached]
cfg.trainer_params["callbacks"] = cbs

### Build Trainer, Model, Optimizer, Scheduler and Lightning Modules

In [9]:
from pytorch_lightning import Trainer
from protera_stability.trainer.lightning_train import DataModule, TrainingLit

# build trainer
print("Building trainer")
trainer_params = {k : instantiate(v) for k, v in cfg.trainer_params.items()}
trainer = Trainer(**trainer_params)

# build model
print("Building models")
model = instantiate(cfg.model)

# build optim
print("Building optimizers")
cfg.optim.params.model = model
optimizer = instantiate(cfg.optim)

# build scheduler
print("Building schedulers")
cfg.scheduler.optimizer = optimizer
scheduler = instantiate(cfg.scheduler)
schedulers = [{"scheduler": scheduler, "interval": "epoch"}]

# build modules
print("Building modules")
module = TrainingLit(cfg, model=model, optimizer=optimizer, schedulers=schedulers)
data_module = DataModule(cfg)

train_dl = data_module.train_dataloader()
print(f"=== USING {cfg.experiment.sampling_method} as Sampling Method ===")
print(f"=== USING {len(train_dl.sampler)} out of {len(train_dl.dataset)} samples ===")

if cfg.experiment.sampling_method == "diversity":
    print(f"=== SIZE WAS DETERMINED BY {train_dl.sampler.stopped_by} ===")

elif cfg.experiment.sampling_method == "random":
    print(f"=== SIZE WAS DETERMINED BY RANDOM PERCENT OF {cfg.experiment.random_percent} ===")

/home/roberto/anaconda3/envs/protera-stability/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory logs/example_all-data/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Building trainer
Building models
Building optimizers
Building schedulers
Building modules


/home/roberto/anaconda3/envs/protera-stability/lib/python3.7/site-packages/pytorch_lightning/utilities/seed.py:57: UserWarning: No correct seed found, seed set to 1455664959
  rank_zero_warn(f"No correct seed found, seed set to {seed}")
Global seed set to 1455664959


=== USING diversity as Sampling Method ===
=== USING 6135 out of 8204 samples ===
=== SIZE WAS DETERMINED BY CUTOFF ===


In [10]:
cfg["experiment"]

{'sampling_method': 'diversity', 'diversity_cutoff': 0.8, 'random_percent': 0.15, 'random_split': 0.8, 'name': 'example_all-data'}

In [ ]:
trainer.fit(module, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type       | Params
----------------------------------------
0 | model    | ProteinMLP | 4.7 M 
1 | train_r2 | R2Score    | 0     
2 | valid_r2 | R2Score    | 0     
3 | test_r2  | R2Score    | 0     
----------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.891    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1455664959
Global seed set to 1455664959


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

### Run as fn

In [ ]:
#cfg, trainer_dict = do_train(cfg)

In [ ]:
#cfg, trainer_dict = do_test(cfg, trainer_dict)

In [ ]:
2048*2